## Running ISR from a notebook

Craig Lage - 16-Jun-22

In [ ]:
import numpy as np
#%matplotlib ipympl
import matplotlib.pyplot as plt
import astropy.io.fits as pf
from lsst.ip.isr import IsrTask, IsrTaskConfig
from lsst.daf.butler import Butler

In [ ]:
butler = Butler('/repo/embargo', collections=["LATISS/raw/all", "LATISS/calib"])

In [ ]:
isrConfig = IsrTaskConfig()
isrConfig.doLinearize=False
isrConfig.doOverscan=True
isrConfig.doAssembleCcd=True
isrConfig.doBias=True
isrConfig.doVariance=True
isrConfig.doLinearize=False
isrConfig.doCrosstalk=False
isrConfig.doBrighterFatter=False
isrConfig.doDark=False
isrConfig.doStrayLight=False
isrConfig.doFlat=False
isrConfig.doFringe=False
isrConfig.doApplyGains=False
isrConfig.doDefect=False
isrConfig.doNanMasking=True
isrConfig.doInterpolate=False
isrConfig.doSaturation=False
isrConfig.doSaturationInterpolation=False


# Adjust these as needed and add as many more as you want

In [ ]:
isrTask = IsrTask(config=isrConfig)

In [ ]:
expId = 2022121200745
exp = butler.get('raw', detector=0, exposure=expId)
biasExp = butler.get('bias', detector=0, exposure=expId)

In [ ]:
isrResult = isrTask.run(exp, bias=biasExp)

In [ ]:
# Now look at the data with matplotlib

from matplotlib.colors import LogNorm

def colorbar(mappable):
    from mpl_toolkits.axes_grid1 import make_axes_locatable
    last_axes = plt.gca()
    ax = mappable.axes
    fig = ax.figure
    divider = make_axes_locatable(ax)
    cax = divider.append_axes("right", size="5%", pad=0.05)
    cbar = fig.colorbar(mappable, cax=cax)
    plt.sca(last_axes)
    return cbar

plt.figure(figsize=(8,8))
plt.suptitle(f"Image",fontsize=18)
arr = isrResult.exposure.image.array[100:200,100:200]
img = plt.imshow(arr, norm=LogNorm(vmin=10, vmax=1000), interpolation='Nearest', cmap='gray')
colorbar(img)
plt.tight_layout(h_pad=1)


In [ ]:
import lsst.afw.display
import lsst.display.astrowidgets
lsst.afw.display.setDefaultBackend("astrowidgets")

In [ ]:
# Create a display object:
display = lsst.afw.display.Display(dims=(800, 600))  # size in screen pixels
display.embed()

In [ ]:
display.mtv(isrResult.exposure)

Useful Keyboard Shortcuts.
Ginga supplies interactive keyboard commands as listed in the table below. The left set should always be active. The right set need to have the appropriate mode selected by typing space-. More details available at https://ginga.readthedocs.io/en/stable/quickref.html

Key	Action		Mode	Key	Action
`	Recenter and zoom to fit.		Space-d		Distribution mode:
{1-0}	Zoom to #x scale.			b, Up	Previous distribution function.
S-{1-0}	Zoom to 1/#x scale.			n, Down	Next distribution function.
p	Recenter image to mouse location.		Space-s		Cut mode:
c	Recenter to image center.			b, Up	Previous cut function.
Middle mouse	Recenter image to mouse location.			n, Down	Next cut function.
[	Horizontal flip.			Left mouse	Drag to set high/low cut levels.
]	Vertical flip.			C-Left mouse	Drag to set only low cut level.
\	Transpose image.			S-Left mouse	Drag to set only high cut level.
R	Reset image to default orientation.			h	Set high cut level to current pixel value.
a	Auto-contrast.			l	Set low cut level to current pixel value.
D	Reset color distribution to linear.			S	Set cut levels to min/max.
T	Reset contrast to default.		Space-t		Contrast mode:
Y	Reset colormap to grayscale.			Left mouse	Drag to set contrast and offset.
I	Invert colormap.		Space-y		Colormap mode:
b, Up	Previous color map.
n, Down	Next color map.
c	Toggle color bar.
Space-r		Rotation mode:
Left mouse	Free rotation about current center.

In [ ]:
mData = butler.get('raw.metadata', detector=0, exposure=expId)

In [ ]:
for key in mData.keys():
    print(key, mData[key])